## Mirror SmugMug Statistics

This notebook runs queries against the `sqlite` database `mirror.db` that holds consolidated SmugMug online image metadata.

Images are here: [https://conceptcontrol.smugmug.com/](https://conceptcontrol.smugmug.com/).

In [1]:
NB. J version - notebook runtime
smoutput 9!:14''
smoutput timestamp 6!:0 ''


j903/j64avx2/windows/release-a/commercial/www.jsoftware.com/2021-12-16T15:15:09/clang-13-0-0/SLEEF=1
01 Dec 2022 09:47:34


### Load `mirrorstats` script

The [`mirrorstats.ijs`](https://github.com/bakerjd99/jacks/blob/master/mirrorxref/mirrorstats.ijs) 
script must be made available to the J `load` utility.  The [`JOD`](https://analyzethedatanotthedrivel.org/the-jod-page/) `mls` verb does this automatically. 

Alternatively you can edit the J start up script `~config/startup.ijs` and add lines like:

     buildpublic_j_ 0 : 0
     mirrorstats  ~temp/mirrorstats.ijs
     )

In [2]:
load 'mirrorstats'

NB. (mirrorstats) interface word(s): 20221130j114020
NB. -------------------------------
NB. NotDivisible        NB. albums with image counts that are not divisible by 3 and 5
NB. albdist             NB. all mean album distances km from position (x)
NB. fsd                 NB. fetch sqlite dictionary array
NB. fst                 NB. fetch sqlite reads table
NB. gpsextremesgallery  NB. list images with gps extremes
NB. meanalbdist         NB. mean km distance of geotagged album images from (x)



### Set `mirror.db` database file location 

**Note:** a test [`mirror.db`](https://github.com/bakerjd99/smugpyter/tree/master/testdata/mirror.zip) is available on GitHub.

In [3]:
NB. set sqlite db file
(3 : 0)''
ALTMIRRORDBPATH_mirrorstats_=:''
if.     IFWIN  do. ALTMIRRORDBPATH_mirrorstats_=:'c:/SmugMirror/Documents/XrefDb/'
elseif. IFUNIX do. ALTMIRRORDBPATH_mirrorstats_=:'~/jupyter_notebooks/dbs/'
end.
if. fexist dbf=: ALTMIRRORDBPATH,MIRRORDB do. dbf else. 'db not set' end.
)
smoutput dbf

c:/SmugMirror/Documents/XrefDb/mirror.db


### Set various utils

In [4]:
NB. set portable box characters for LaTeX
0 0$portchars ''

NB. cell clip format
ccf=: [ {."1 [: ": ]

### Set homebase coordinates

The home coordinates are set in this fashion to avoid embedding location data in code. 

`homebase.txt` is a simple text file with a single line like:

    109.375956 -33.852775


Longitude and Latitude are in decimal degrees and use the sign conventions of Jean Meeus's book [*Astronomical Algorithms*](https://www.amazon.com/Astronomical-Algorithms-Jean-Meeus/dp/0943396611).

In [5]:
NB. longitude and latitude of home decimal degrees
NB. western longitudes +, northern latitudes +
(3 : 0)''
if. fexist  ALTMIRRORDBPATH,'homebase.txt' do. 
  MeeusHomeLonLat_mirrorstats_=: _999&".read ALTMIRRORDBPATH,'homebase.txt'
else.
  MeeusHomeLonLat_mirrorstats_=: 0 0
end.
)
smoutput (;:'Longitude Latitude') ,: <"0 MeeusHomeLonLat_mirrorstats_

+---------+--------+
|Longitude|Latitude|
+---------+--------+
|0        |0       |
+---------+--------+


### Load SQLITE data

In [6]:
NB. image columns
({."1 r)=: {:"1 r=: 'select * from OnlineImage order by UpLoadDate desc' fsd dbf

NB. album columns
({."1 r)=: {:"1 r=: 'select * from Album order by OnlineImageCount desc' fsd dbf

### Total online image count

In [7]:
'select count(*) as OnlineCount from OnlineImage' fst dbf

+-----------+
|OnlineCount|
+-----------+
|5468       |
+-----------+


### Recent album adjustments

In [8]:
NB. most recently "updated" albums
r=: 'select ImagesLastUpdated, AlbumName from Album order by LastUpdated desc limit 5' fsd dbf

NB. display with album "centroid" distance
NB. 0 distance means no geotagged images
(>&.> {:"1 r) , < ,. meanalbdist&> ;1{1{ r

+-------------------------+--------------------------+-------+
|2022-11-30T19:47:07+00:00|GPS Extremes              |  12849|
|2022-11-30T17:00:12+00:00|Australia New Zealand 2022|15106.6|
|2022-11-30T17:08:07+00:00|Panoramas                 |11817.4|
|2022-11-27T20:46:36+00:00|Image Hacking             |  11622|
|2022-11-26T21:44:04+00:00|Alpha Layered             |      0|
+-------------------------+--------------------------+-------+


### Recent image additions

In [9]:
NB. last 15 images loaded
;"1 ] 15 {. (10 {.&.> RealDate) ,. (<' - ') ,. OnlineImageFile

2022-10-20 - walpa gorge rock trail.jpg               
2022-10-20 - kata tjuta morning.jpg                   
2022-10-20 - conglomerate tjuta boulder.jpg           
2022-10-29 - old cars ship dock.jpg                   
2022-10-19 - me uluru sunset.jpg                      
2022-11-06 - color pipe dream.jpg                     
2022-11-27 - receding.jpg                             
2022-11-27 - circle bin chicken saint.png             
2022-10-27 - mali christchurch bridge of rembrance.jpg
2022-11-06 - bondi beach swim club.jpg                
2022-10-31 - victoria albert park auckland.jpg        
2022-10-31 - auckland downtown dock.jpg               
2022-11-06 - bondi mirror ball.jpg                    
2022-10-22 - mali cruise ship opera house.jpg         
2022-10-22 - ok quay cruise ship.jpg                  


### Album statistics

In [10]:
NB. 10 largest albumns
10 {. (>AlbumName) ,. ": ,. >OnlineImageCount

Idaho Instants                487
The Way We Were               430
Cell Phoning It In            354
Weekenders                    336
Missouri Moments              315
California Captures           303
My Kids                       198
Australia New Zealand 2022    185
Image Hacking                 178
Panoramas                     170


In [11]:
NB. album image counts
dstat OnlineImageCount

sample size:          85
minimum:               0
maximum:             487
1st quartile:         16
2nd quartile:         48
3rd quartile:        109
first mode:            0
first antimode:      487
mean:            78.3176
std devn:        97.0931
skewness:         2.2279
kurtosis:         8.2003


### Images per year

My image dates are a mixture of EXIF dates and manually assigned dates. The manually assigned dates are rarely accurate and reflect my best guess as to when a picture was taken. Many images do not have dates. They show up in year 0.

In [12]:
NB. images per year - year 0 means no date
70 list ',' ,.~  ": rf=: \:~ |: yic=: > freq 0 {"1 rdt=: itYMDhms RealDate

2022 508, 2021 164, 2020  98, 2019 446, 2018 294, 2017 227, 2016 298, 
2015 207, 2014  98, 2013 172, 2012 178, 2011 217, 2010 152, 2009 144, 
2008 180, 2007 284, 2006 166, 2005 120, 2004 110, 2003  79, 2002 118, 
2001 136, 2000  34, 1999   6, 1996   1, 1995  19, 1994   8, 1993   8, 
1992  18, 1991  13, 1990   4, 1989  14, 1988   6, 1987  10, 1986   6, 
1985   4, 1984   2, 1983  42, 1982   4, 1981  37, 1980  17, 1979  90, 
1978  27, 1977  10, 1976  20, 1975   8, 1974  22, 1973   4, 1972   5, 
1971  23, 1970  33, 1969  22, 1968  45, 1967  32, 1966  24, 1965   5, 
1964  13, 1963   9, 1962  21, 1961   6, 1960   7, 1959  17, 1958  14, 
1957   7, 1956   6, 1955  11, 1954  10, 1953   4, 1952   7, 1951   3, 
1950   9, 1949  18, 1948  10, 1947   3, 1946   5, 1945   2, 1944   2, 
1943   1, 1942   1, 1941   7, 1940   6, 1939   2, 1938   3, 1937   2, 
1936   5, 1935   4, 1934   3, 1933   1, 1932   1, 1931   5, 1930   1, 
1929   1, 1928   1, 1927   2, 1925   2, 1924   1, 1920   2, 1919   2, 
1917  

In [13]:
NB. compare current year image count to prior years
NB. show no more than 5 greater counts
gyc=: rf #~ (1 {"1 rf) > cyc=: (<0;1){rf
(5 <. #gyc) {. gyc ,. (1 {"1 gyc) - cyc

Images per year statistics since the year 2000. It was around 2000 that I started saving digital image files.

In [14]:
NB. images per year statistics for years after 1999
dstat (1 {"1 rf) #~ 1999 < 0 {"1 rf

sample size:          23
minimum:              34
maximum:             508
1st quartile:        118
2nd quartile:        166
3rd quartile:        227
first mode:           98
first antimode:      508
mean:            192.609
std devn:        112.527
skewness:         1.3617
kurtosis:         4.5651


### Uploads per year

I often upload old images that get counted in prior years. These counts reflect actual database inserts. Note the count of 724 in 2019. I did not insert that many images in 2019. The upload count probably reflects reorganizing existing images. The count of 1531 in 2009 marks the year I transferred my online images to SmugMug.

In [15]:
70 list ',' ,.~  ": uf=: \:~ |: upc=: > freq 0 {"1 udt=: itYMDhms UploadDate

2022  529, 2021  203, 2020  109, 2019  724, 2018  352, 2017  319, 
2016  362, 2015  222, 2014  102, 2013  256, 2012  233, 2011  314, 
2010  212, 2009 1531,                                             


In [16]:
NB. uploads per year statistics - less outliers > 600
dstat (1 {"1 uf) #~ 600 > 1 {"1 uf

sample size:         12
minimum:            102
maximum:            529
1st quartile:     207.5
2nd quartile:     244.5
3rd quartile:     335.5
first mode:         529
first antimode:     529
mean:            267.75
std devn:        117.71
skewness:        0.6032
kurtosis:        3.2142


### *Flarp* rate

The *flarp rate* gives a rough indication of how much image rework and reloading I do. It shows up as a discrepancy between year counts and upload counts. 

In [17]:
smoutput flc=: (0{upc) ,: (1{upc) - 1{(\:~)&.|: yic #"1~ (0{yic) e. 0{upc

2022 2021 2020 2019 2018 2017 2016 2015 2014 2013 2012 2011 2010 2009
  21   39   11  278   58   92   64   15    4   84   55   97   60 1387


Flarp statistics excluding the first upload year.

In [18]:
dstat }: 1{flc

sample size:          13
minimum:               4
maximum:             278
1st quartile:         18
2nd quartile:         58
3rd quartile:         88
first mode:           21
first antimode:       21
mean:            67.5385
std devn:        70.3108
skewness:         2.1603
kurtosis:         7.3978


### `NotDivisible` counts

`NotDivisible` counts how many images must be added to a gallery so that the total number of gallery images is evenly divisible by 3 and 5. The iPhone SmugMug app displays images in rows of 3 (portrait) or 5 (landscape) and I like completely filled rows. When the total is evenly divisible by 15 = 3 * 5 both portrait and landscape rows are filled.

In [19]:
10 {. NotDivisible dbf

+-------+-+-+--+--------------------+
|[Count]|3|5|15|[Album Name]        |
+-------+-+-+--+--------------------+
|148    |2|2|2 |Simply Signs        |
+-------+-+-+--+--------------------+
|178    |2|2|2 |Image Hacking       |
+-------+-+-+--+--------------------+
|28     |2|2|2 |Alpha Layered       |
+-------+-+-+--+--------------------+
|133    |2|2|2 |Been There Done That|
+-------+-+-+--+--------------------+
|146    |1|4|4 |Caught My Eye       |
+-------+-+-+--+--------------------+
|7      |2|3|8 |Does Not Fit        |
+-------+-+-+--+--------------------+
|7      |2|3|8 |Test Patterns       |
+-------+-+-+--+--------------------+
|7      |2|3|8 |Partial Restorations|
+-------+-+-+--+--------------------+
|22     |2|3|8 |To Much Information |
+-------+-+-+--+--------------------+


### Dominant color keywords

Keywords starting with `0_` are dominant color keys assigned by the 
Python [`SmugPyter`](https://github.com/bakerjd99/smugpyter) program. 

In [20]:
NB. keyword columns
({."1 r)=: {:"1 r=: 'select * from ImageKeywordXr' fsd dbf

In [21]:
topbotk=: 4 : 0
NB.topbotk v-- (x) top and bottom (y) keys
kf=: ofreq y
kf=: (' ' ,. >;0{kf) ,.~ ":>,.;1{kf
(x {. kf) ,. |. (-x) {. kf
)

**Note:** the abundance of neutral, (grey and near grey), colors. Well exposed color balanced images tend, on average, to be neutral. This is why photographers use grey cards and color balancing software picks neutrals.

In [22]:
NB. top/bottom 20 color keywords
ColorKeys=: Keyword #~ ('0_'&-:)@(2&{.)&> Keyword
20 topbotk ColorKeys

542 0_darkslategrey         1 0_forestgreen         
382 0_black                 1 0_aquamarine          
283 0_darkolivegreen        1 0_purple              
250 0_steelblue             1 0_mediumslateblue     
238 0_dimgrey               1 0_magenta             
216 0_darkgrey              1 0_lightyellow         
206 0_rosybrown             1 0_lawngreen           
197 0_cornflowerblue        1 0_lavenderblush       
189 0_lightsteelblue        1 0_indigo              
182 0_gainsboro             1 0_springgreen         
182 0_tan                   1 0_cyan                
176 0_lightgrey             1 0_darkmagenta         
173 0_grey                  1 0_mediumspringgreen   
123 0_silver                1 0_floralwhite         
120 0_lavender              1 0_cornsilk            
 99 0_peru                  1 0_mediumorchid        
 96 0_lightslategrey        2 0_greenyellow         
 96 0_sienna                2 0_lime                
 88 0_saddlebrown           2 0_lightgreen    

### Print size keywords

Print size assumes a resolution of 360 DPI (Dot (pixels) per Inch). This is slightly
higher than the frequent default of 300 DPI and was chosen because twice
the resolution, 720 DPI, is a common setting for fine art prints. Online prints
from SmugMug and elsewhere that use the print sizes computed here will render well.

In [23]:
NB. top/bottom 20 print size keywords
SizeKeys=: ColorKeys -.~ Keyword #~ ({.&> Keyword) e.&> <'0123456789'
NB. remove odd keys that do not follow name conventions
NssKeys=: <;._1 ' 0z0 0z1'
SizeKeys=: SizeKeys -. (<;._1 ' 50th'),NssKeys
20 topbotk SizeKeys

995 5x7       1 2.25x4  
951 4x6       1 10x16   
575 8x12      1 8x40    
347 10x15     1 12x30   
298 6x8       1 16x24   
274 5x6.7     1 3x3     
199 3x4       1 5x8     
154 2.5x3.5   1 5.5x14  
139 4x5       1 9x21    
 94 5x5       1 4x5.3   
 67 6x10      1 2x2.5   
 63 8x10      2 10x40   
 61 3x5       2 16x20   
 52 2x3       2 12x24   
 41 10x14     2 10x13   
 28 4x4       2 11x28   
 28 8x8       2 11x16   
 25 3.5x5     2 9x16    
 18 6x6       2 3.5x3.5 
 18 4.5x8     3 6x24    


In [24]:
NB. size keys that indicate not enough pixels or nonstandard aspect ratio
NssKeys ,. <"0 +/"1 NssKeys =/ Keyword 

+---+---+
|0z0|267|
+---+---+
|0z1|489|
+---+---+


### Descriptive keywords

All keywords that are not color or size keys.

In [25]:
DescKeys=: Keyword -. ColorKeys,SizeKeys,NssKeys
20 topbotk DescKeys

1456 geotagged                      1 rafting                     
1073 usa                            1 prayer                      
 570 mali                           1 tooth                       
 420 me                             1 saber                       
 259 idaho                          1 columns                     
 235 park                           1 remnants                    
 209 helen                          1 pictographs                 
 202 iphone                         1 skeleton                    
 188 lake                           1 sizes                       
 177 river                          1 rafts                       
 173 canada                         1 dell                        
 168 missouri                       1 large                       
 151 jacob                          1 josie                       
 148 museum                         1 powers                      
 129 montana                        1 fremont                 

### Panorama statistics

I have enjoyed making panoramas for years. I consider any image with an *`aspect ratio >: 2`* a panorama.  Many of my panoramas have nonstandard ratios, i.e. they are not simple `2x1`, `3x1`, `4x1`, `5x1`  ... ratios and consequently do not get assigned a size key. 

In [26]:
NB. rounded image aspect ratios - aspect ratio is always >: 1
ratios=: 0.005 round %/"1 \:"1~ OriginalHeight ,. OriginalWidth

NB. aspect ratio counts
PanAspects=: >: i. >: >. >./ratios
smoutput RatioCounts=: ;"1 (2 <\ PanAspects) ,. <"0 }. }: PanAspects histogram2 ratios

1 2 5208
2 3  165
3 4   66
4 5   25
5 6    4
6 7    0


### Total number of panoramas with *Aspect Ratio >: 2*

**Note:** this simple criteria of picking images based on aspect ratio misses some panoramas. Some pictures, built from more than one image, have standard aspect ratios.

In [27]:
+/pancnts=: }. 2 {"1 RatioCounts

260


### Online panorama ratios

In [28]:
NB. percentage of panoramic images by aspect ratio - last % is overall
smoutput panper=: (}. 0 2 {"1 RatioCounts) ,. 0.01 round 100 * pancnts % #ratios
smoutput '-' ,: '%' ,~ ": }. +/panper

2 165 3.02
3  66 1.21
4  25 0.46
5   4 0.07
6   0    0
---------
260 4.76%


### Most extreme panoramas *Aspect Ratio >: 5*

In [29]:
NB. most extreme panoramas, i.e. aspect ratios >: 5
80 ccf (OnlineImageFile ,. RealDate ,. Keywords) #~ 5 <: ratios

+---------------------------------------+-------------------+-------------------
|pyramid lake panorama.jpg              |2021-05-04T09:37:47|0_dodgerblue; 8x40;
+---------------------------------------+-------------------+-------------------
|adndlbanner01.jpg                      |                   |0_gold; 0z0; adndlb
+---------------------------------------+-------------------+-------------------
|jodlittlecubebanner.jpg                |                   |0_lightgrey; 0z0; j
+---------------------------------------+-------------------+-------------------
|alexandra bridge ottawa [407055261].jpg|2007-02-28T14:10:00|0_gainsboro; 0z1; a
+---------------------------------------+-------------------+-------------------


### Recently uploaded panoramas

In [30]:
NB. most recent panoramas
80 ccf ,. 5 {. (OnlineImageFile ,. RealDate ,. Keywords) #~ 2 <: ratios

+-----------------------------------------+-------------------+-----------------
|kata tjuta morning.jpg                   |2022-10-20T06:10:49|0_lightgreen; 8x3
+-----------------------------------------+-------------------+-----------------
|color pipe dream.jpg                     |2022-11-06T12:23:50|0_chocolate; 0z1;
+-----------------------------------------+-------------------+-----------------
|auckland downtown dock.jpg               |2022-10-31T15:19:05|0_cornflowerblue;
+-----------------------------------------+-------------------+-----------------
|sydney darling harbor panorama.jpg       |2022-11-03T15:56:30|0_royalblue; 6x24
+-----------------------------------------+-------------------+-----------------
|canberra national arboretum hill view.jpg|2022-10-15T17:16:55|0_darkgreen; 8x20
+-----------------------------------------+-------------------+-----------------


### GPS distances

In [31]:
NB. distances in km from near to far excluding (0,0) lb
NB. convert to Meeus lb convention for (earthdist)
plb=: (i. #Longitude) ,. (-Longitude) ,. Latitude
plb=: |: plb #~ -.0 0 -:"1 ] 1 2 {"1 plb
plb=: /:~ (0{plb) ,.~ MeeusHomeLonLat earthdist 1 2{plb

In [32]:
NB. format image distance
fmdist=: {{;"1 ":&.> (<"0 ] 0.01 round 0 {"1 y) ,. (<' - ') ,. (1 {"1 y){OnlineImageFile}}

In [33]:
NB. 10 nearest geotagged images km
fmdist 10{.plb

1042.84 - ghana secondary school teacher bungalows 1976.jpg  
1042.93 - corn harvest students tamale ghana.jpg             
1043.05 - ghana secondary school [12075167].jpg              
3446.24 - me livingstone airport [47004492].jpg              
3453.59 - mali sprayview hotel [51064919].jpg                
3454.27 - curio market from balloon [49529252].jpg           
3454.37 - vic falls train station [51228396].jpg             
3454.71 - me livingstone statue [19983766].jpg               
3454.82 - victoria falls hotel distance marker [36406766].jpg
3463.73 - eclipse tour group [2486870].jpg                   


In [34]:
NB. 10 furthest geotagged images km
fmdist |. _10{.plb

18358.9 - ruth kwajalein airport sign 1981.jpg           
17705.9 - lab south view 12x4 [5037801].jpg              
17705.9 - mid pacific marine lab from air [254442016].jpg
17705.9 - me next atoll looking south [5088708].jpg      
17704.7 - enewetak from air [5561415].jpg                
17694.4 - me runit dome [6671846].jpg                    
17687.9 - deep diving [7444953].jpg                      
16654.9 - me photographing battleship missouri.jpg       
16654.8 - nimitz statue missouri battleship.jpg          
16654.8 - battleship missouri plaque.jpg                 


In [35]:
NB. image distance statistics
dstat 0 {"1 plb

sample size:        1458
minimum:         1042.84
maximum:         18358.9
1st quartile:    10065.9
2nd quartile:    11879.8
3rd quartile:    12125.1
first mode:      11909.8
first antimode:  1042.84
mean:            11546.2
std devn:        2187.21
skewness:         _0.511
kurtosis:         6.0708


In [36]:
NB. 1s mark panoramas
pbits=: (}.PanAspects) <:"0 1 %/"1 \:"1~ OriginalHeight ,. OriginalWidth

NB. panorama image distances
gtp=: plb #~ (1 {"1 plb) e. I. 0{pbits 

In [37]:
NB. 10 nearest panoramas
fmdist 10 {. gtp

3549.61 - ndeke hotel sign [37985773].jpg           
7656.21 - me spittal pond bermuda [278786543].jpg   
8879.8 - 132 topham terrace orleans white winter.jpg
8881.84 - petrie island river [98449654].jpg        
8897.37 - alexandra bridge ottawa [407055261].jpg   
8928.88 - battersea beach dog lake panorama.jpg     
8933.55 - kingston from fort henry [2355485].jpg    
8934.79 - confederation harbor [12870536].jpg       
9429.28 - bed and breakfast view.jpg                
9682.96 - artsgarden.jpg                            


In [38]:
NB. 10 furthest panoramas
fmdist |. _10 {. gtp

17705.9 - lab south view 12x4 [5037801].jpg      
16653.2 - waikiki beach diamond head panorama.jpg
16651.5 - honolulu from diamond head panorama.jpg
16651.3 - diamond head caldera interior.jpg      
16564.4 - mauna kea sunset summit panorama.jpg   
16055.1 - ship bridge views.jpg                  
15889.9 - auckland downtown dock.jpg             
15830.7 - brisbane river story bridge.jpg        
15227.9 - first day at sea.jpg                   
15212.1 - color pipe dream.jpg                   


In [39]:
NB. mean panorama distance
dstat 0 {"1 gtp

sample size:         128
minimum:         3549.61
maximum:         17705.9
1st quartile:    11651.4
2nd quartile:    11975.9
3rd quartile:    12369.5
first mode:      12101.2
first antimode:  3549.61
mean:            12150.5
std devn:        1926.02
skewness:        _0.0747
kurtosis:          6.216


In [40]:
NB. mean album geotagged image distances
mad=: albdist 0

In [41]:
NB. album "centroid" distance statistics
dstat ;0 {"1 mad

sample size:          63
minimum:         1042.94
maximum:         17794.8
1st quartile:    9633.19
2nd quartile:    11373.3
3rd quartile:    11935.2
first mode:      12488.9
first antimode:  12488.9
mean:              10533
std devn:        2719.14
skewness:        _0.8158
kurtosis:          5.426


In [42]:
NB.format album distances.
fmtad=:{{('8.2'&(8!:2) ,. km) ,. ' - ' ,"1 alb [ 'km alb'=. |: x {. y}}

In [43]:
NB. 10 nearest albums
10 fmtad /:~ mad

 1042.94 - Ghana 1970's                  
 3596.33 - Zambia Eclipse Trip           
 4678.38 - ACS School Trips 1960's       
 5310.90 - Beirut Lebanon 1960's         
 6378.44 - Iran 1960's                   
 6735.58 - Barbados Late 1970's          
 6736.08 - Diving at Bellairs Barbados BW
 6884.68 - South America 1979            
 7657.01 - Briefly Bermuda               
 8665.71 - New York 2005                 


In [44]:
NB. 10 furthest albums
10 fmtad \:~ mad

17794.80 - Enewetak Atoll 1980's     
16610.10 - Hawaii 2018               
15106.57 - Australia New Zealand 2022
12848.99 - GPS Extremes              
12604.47 - 63                        
12597.30 - California Captures       
12488.94 - Oregon Ogling             
12362.69 - Seattle Vancouver 2020    
12249.74 - Inlaws Outlaws and Friends
12218.38 - Along the Yukon River     


### GPS Extremes Gallery

A collection of geotagged images that are extreme in the sense of:

1. Near and far the (0,0) origin in km
2. Highest elevations in meters
3. Near and far the equator by latitude
4. Near and far the prime meridian by longitude
5. Most northern and southern latitudes
6. Most eastern and western longitudes

**Note:** The altitude data in the SQLite database are all positive numbers. This is usually fine as most images are above sea level but a few rare images are below sea level. Camera metadata indicates this with an above or below sea level flag but this flag is not collected.

In [45]:
NB. GPS extremes 
smoutput #gx=: gpsextremesgallery dbf
80 ccf ;"1 (' | '&,)@":&.> gx

30
 | qbXqVgC | ghana secondary school teacher bungalows 1976.jpg | 9.39475 _0.8176
 | K7JKbs8 | corn harvest students tamale ghana.jpg | 9.39566 _0.816272 0 | 1049
 | kNRs3X8 | ghana secondary school [12075167].jpg | 9.39672 _0.816726 0 | 1049.
 | vSNN8DC | me livingstone airport [47004492].jpg | _17.8228 25.8186 0 | 3439.0
 | mBXdmNg | me hampton court 1968 [992782789].jpg | 51.4038 _0.339633 0 | 5722.
 | 6fzX7cV | me ruth miraflores locks panama 1979.jpg | 8.99543 _79.5899 0 | 885
 | Nmc5SNX | me mali mount evans sign.jpg | 39.5877 _105.642 4313 | 11341.7     
 | 56MR5dM | mount evans western panorama.jpg | 39.5883 _105.644 4358 | 11341.8 
 | QBMjK2B | dawson yukon street panorama 2002.jpg | 64.0601 _139.435 0 | 12174.
 | dRXXhPb | yukon at dawson at dusk [163426977].jpg | 64.0561 _139.441 0 | 1217
 | 8gvcsMV | tied canoes [3559560].jpg | 62.8281 _138.507 0 | 12240.3           
 | 8bHbZBX | me yukon river sunset canada [124957718].jpg | 62.8277 _138.512 0 |
 | WCSbvvR | mali larnach

### Geotagged Image Waypoints GPX

Compute GPX file for geotagged images. This file can be used to set waypoints in various GPS devices.

In [46]:
load 'gpxutils'

NB. (gpxutils) interface word(s): 20220916j141736
NB. -----------------------------
NB. allrecent    NB. all recent images from last waypoint generation
NB. csvfrwpt     NB. poi CSV text from waypoint text file
NB. gpxfrmapkml  NB. gpx from Google maps kml
NB. gpxfrmirror  NB. extracts geotagged images from mirror_db and generates gpx
NB. gpxfrpoicsv  NB. converts poi csv files to gpx
NB. gpxfrrecent  NB. gpx from recent waypoints



In [47]:
NB. waypoints gpx
gpx=: gpxfrmirror dbf

NB. save in working directory - requires configured J folder
smoutput gpxfile=: jpath '~JUPYTER/dbs/geotagged_images.gpx'
(toHOST gpx) write gpxfile

c:/users/baker/jupyter_notebooks/dbs/geotagged_images.gpx


### All done - thanks for playing!